# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [2]:
# Library utama
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn: preprocessing, model, evaluasi
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [5]:
# Load dataset hasil clustering yang telah didapat sebelumnya
df = pd.read_csv('df_final_result.csv')

# Cek beberapa baris awal
df.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Cluster
0,14.0,73.0,9.5,82.0,3,1010.82,2.0,3,3.5,1,1
1,39.0,96.0,8.5,71.0,3,1011.43,7.0,1,10.0,1,3
2,30.0,64.0,7.0,16.0,0,1018.72,5.0,1,5.5,2,2
3,38.0,83.0,1.5,82.0,0,1026.25,7.0,1,1.0,0,1
4,27.0,74.0,17.0,66.0,2,990.67,1.0,3,2.5,2,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  float64
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  int64  
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index              13200 non-null  float64
 7   Season                13200 non-null  int64  
 8   Visibility (km)       13200 non-null  float64
 9   Location              13200 non-null  int64  
 10  Cluster               13200 non-null  int64  
dtypes: float64(7), int64(4)
memory usage: 1.1 MB


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [7]:
# Pisahkan fitur dan label target, yang mana Cluster akan berperan sebagai label
X = df.drop(columns=['Cluster'])  # fitur, mengecualikan cluster
y = df['Cluster']                 # label target hasil clustering

# Deteksi fitur numerik dan kategorikal
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# Bagi data menjadi training dan testing (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [9]:
# Pipeline preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Pipeline model dengan Random Forest
clf_pipeline = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Train model
clf_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Temperature', 'Humidity',
                                                   'Wind Speed',
                                                   'Precipitation (%)',
                                                   'Cloud Cover',
                                                   'Atmospheric Pressure',
                                                   'UV Index', 'Season',
                                                   'Visibility (km)',
                                                   'Location']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  [])])),
                ('classifier', RandomForestClassifier(random_state=42))])

Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [15]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# Prediksi
y_pred = clf_pipeline.predict(X_test)

# Evaluasi
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')  # karena multiclass

print(f" Accuracy: {acc:.4f}")
print(f" F1-Score: {f1:.4f}")
print("\n Classification Report:\n", classification_report(y_test, y_pred))
print(" Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

 Accuracy: 0.9667
 F1-Score: 0.9667

 Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.99      0.92        95
           1       0.97      0.96      0.97      1126
           2       0.98      0.98      0.98      1125
           3       0.94      0.91      0.93       294

    accuracy                           0.97      2640
   macro avg       0.94      0.96      0.95      2640
weighted avg       0.97      0.97      0.97      2640

 Confusion Matrix:
 [[  94    0    1    0]
 [   8 1086   23    9]
 [   6    8 1103    8]
 [   1   22    2  269]]


Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [12]:
from sklearn.model_selection import GridSearchCV

# Parameter Grid untuk Random Forest
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5]
}

# Wrap pipeline dengan GridSearchCV
grid_search = GridSearchCV(clf_pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)

# Train dengan Grid Search
grid_search.fit(X_train, y_train)

# Tampilkan hasil terbaik
print(f"Best Params: {grid_search.best_params_}")
print(f"Best Score: {grid_search.best_score_:.4f}")


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Params: {'classifier__max_depth': 20, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 200}
Best Score: 0.9699


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [13]:
# Gunakan model terbaik dari GridSearch
best_model = grid_search.best_estimator_
y_pred_tuned = best_model.predict(X_test)

# Evaluasi ulang
acc_tuned = accuracy_score(y_test, y_pred_tuned)
f1_tuned = f1_score(y_test, y_pred_tuned, average='weighted')

print(f"\n Accuracy Setelah Tuning: {acc_tuned:.4f}")
print(f" F1-Score Setelah Tuning: {f1_tuned:.4f}")
print("\n Classification Report Setelah Tuning:\n", classification_report(y_test, y_pred_tuned))
print(" Confusion Matrix:\n", confusion_matrix(y_test, y_pred_tuned))



 Accuracy Setelah Tuning: 0.9667
 F1-Score Setelah Tuning: 0.9667

 Classification Report Setelah Tuning:
               precision    recall  f1-score   support

           0       0.87      0.98      0.92        95
           1       0.98      0.96      0.97      1126
           2       0.98      0.98      0.98      1125
           3       0.93      0.93      0.93       294

    accuracy                           0.97      2640
   macro avg       0.94      0.96      0.95      2640
weighted avg       0.97      0.97      0.97      2640

 Confusion Matrix:
 [[  93    1    1    0]
 [   7 1081   26   12]
 [   6    6 1106    7]
 [   1   20    1  272]]


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

In [14]:
print("PERBANDINGAN:")
print(f"Before Tuning  -> Accuracy: {acc:.4f}, F1-Score: {f1:.4f}")
print(f"After Tuning   -> Accuracy: {acc_tuned:.4f}, F1-Score: {f1_tuned:.4f}")

PERBANDINGAN:
Before Tuning  -> Accuracy: 0.9667, F1-Score: 0.9667
After Tuning   -> Accuracy: 0.9667, F1-Score: 0.9667


**Penjelasan**

Berdasarkan pembuatan model machine learning ini ada beberapa hal yang bisa dibahas:
1. Model machine learning ini sudah akurat hingga lebih dari 95%
2. Keseimbangan kelas yang cukup baik, bahkan untuk kelas yang kecil (cluster 0) sekalipun
3. Tidak terlihat adanya overfitting ataupun underfitting. Kenapa? Karena hasilnya (Accuracy, F-1 Score, Precision, and Support) yang cenderung stabil dan tinggi
4. Kekurangan dari model ini adalah ketidakseimbangan label yang mana pada kluster 0 (<100). Hal yang bisa dilakukan selanjutnya adalah dengan menggunakan penyesuaian bobot pada setiap cluster.  